WCS를 이용하여 fits 파일의 특정 픽셀의 적위/적경을 알아내고 simbad/nasa explonet archive 에서 별에 대한 정보를 불러오는 기능.
WCS(World Coordinate System)는 천문학에서 천체 사진의 픽셀 좌표를 실제 천체의 적경(Right Ascension, RA)과 적위(Declination, Dec)로 변환하기 위해 사용하는 좌표 변환 체계이다.

In [ ]:
%pip install astropy astroquery ipywidgets matplotlib ipympl

In [ ]:
%matplotlib widget 

In [ ]:
#매직 커맨드 실행해야 jupyterlab에서 위젯이 제대로 표시된다

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
from astroquery.simbad import Simbad
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib.pyplot as plt
from astropy.visualization import simple_norm
import io
from ipywidgets import FileUpload, Button, VBox
from IPython.display import display


# 전역 변수: 업로드된 파일 정보를 저장
uploaded_file_info = {}

def get_ra_dec_from_pixel(fits_file, x, y):
    """
    FITS 파일의 WCS 정보를 이용해 지정한 픽셀 (x, y)를 하늘 좌표 (RA, Dec)로 변환합니다.
    
    Parameters:
        fits_file: FITS 파일 경로 (str) 또는 파일 객체 (BytesIO)
        x, y (float): 이미지 상의 픽셀 좌표 (0-indexed)
    
    Returns:
        tuple: (RA, Dec) in degrees
    """
    hdul = fits.open(fits_file)
    header = hdul[0].header
    wcs = WCS(header)
    # origin=0: 0-indexed 좌표계 사용
    ra, dec = wcs.all_pix2world(x, y, 0)
    hdul.close()
    return ra, dec

def query_simbad_by_ra_dec(ra, dec, radius=5*u.arcsec):
    """
    SIMBAD 데이터베이스에서 주어진 좌표 근처의 천체 기본 정보를 조회합니다.
    
    Parameters:
        ra (float): 적경 (degrees)
        dec (float): 적위 (degrees)
        radius (Quantity): 검색 반경 (기본 5 arcsec)
        
    Returns:
        astropy.table.Table: SIMBAD 검색 결과 테이블
    """
    coord = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, frame='icrs')
    result = Simbad.query_region(coord, radius=radius)
    return result

def get_exoplanet_info(star_name):
    """
    주어진 별 이름으로 NASA Exoplanet Archive에서 외계 행성 정보를 조회합니다.
    
    Parameters:
        star_name (str): 별의 이름 (SIMBAD에서 얻은 MAIN_ID)
    
    Returns:
        astropy.table.Table or None: 외계 행성 정보 테이블 (없으면 None)
    """
    exoplanets = NasaExoplanetArchive.query_planet(star=star_name)
    return exoplanets

def interactive_get_pixel(fits_file):
    """
    FITS 파일을 읽어 이미지를 표시한 후, 사용자가 마우스로 클릭한 위치의 픽셀 좌표를 반환합니다.
    
    Parameters:
        fits_file: FITS 파일 경로 (str) 또는 파일 객체 (BytesIO)
    
    Returns:
        tuple: (x, y) 픽셀 좌표
    """
    hdul = fits.open(fits_file)
    header = hdul[0].header
    data = hdul[0].data
    wcs = WCS(header)
    hdul.close()
    
    norm = simple_norm(data, 'sqrt', percent=99)
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection=wcs)
    ax.imshow(data, origin='lower', cmap='gray', norm=norm)
    ax.set_xlabel('RA')
    ax.set_ylabel('Dec')
    ax.set_title('마우스로 클릭하여 픽셀 위치를 선택하세요')
    plt.draw()      # 강제로 그리기
    plt.pause(0.1)  # 잠깐 대기해서 그림이 업데이트되도록 함
    # ginput() 함수로 클릭 이벤트를 기다림 (단일 클릭)
    print("이미지에서 원하는 위치를 클릭하세요...")
    pts = plt.ginput(1, timeout=-1)
    plt.close(fig)
    
    if pts:
        x, y = pts[0]
        return x, y
    else:
        print("선택된 좌표가 없습니다.")
        return None, None

def process_fits_file(fits_file):
    """
    업로드된 FITS 파일을 처리하는 전체 작업을 수행합니다.
    1. 이미지에서 마우스 클릭으로 픽셀 좌표 선택
    2. 해당 좌표의 RA, Dec 계산
    3. SIMBAD 및 NASA Exoplanet Archive에서 천체 정보 조회
    """
    # 1. 인터랙티브하게 클릭하여 픽셀 좌표를 얻음
    x, y = interactive_get_pixel(fits_file)
    if x is None or y is None:
        print("픽셀 좌표가 선택되지 않았습니다. 종료합니다.")
        return

    # 2. 선택한 픽셀 좌표를 하늘 좌표(RA, Dec)로 변환
    ra, dec = get_ra_dec_from_pixel(fits_file, x, y)
    print("선택된 픽셀 ({:.2f}, {:.2f}) -> RA: {:.6f} deg, Dec: {:.6f} deg".format(x, y, ra, dec))
    
    # 3. SIMBAD 조회
    simbad_result = query_simbad_by_ra_dec(ra, dec)
    if simbad_result is None or len(simbad_result) == 0:
        print("해당 좌표에서 SIMBAD 결과를 찾을 수 없습니다.")
    else:
        print("SIMBAD 쿼리 결과:")
        print(simbad_result)
        raw_name = simbad_result['MAIN_ID'][0]
        star_name = raw_name.decode('utf-8') if isinstance(raw_name, bytes) else raw_name
        print("선택된 별의 이름 (MAIN_ID):", star_name)
        
        # 4. NASA Exoplanet Archive 조회
        exoplanet_data = get_exoplanet_info(star_name)
        if exoplanet_data is None or len(exoplanet_data) == 0:
            print("해당 별에 대한 외계 행성 정보가 없습니다.")
        else:
            print("외계 행성 정보:")
            print(exoplanet_data)

def on_upload_change(change):
    """
    파일 업로드가 발생하면 업로드된 파일 정보를 저장하고, 메시지를 출력합니다.
    """
    if change['new']:
        # change['new']가 tuple로 전달되는 경우, 각 요소는 업로드된 파일 정보를 담은 dict입니다.
        for filedata in change['new']:
            uploaded_file_info['filename'] = filedata['name']
            uploaded_file_info['content'] = filedata['content']
            print(f"{filedata['name']} 파일이 업로드 되었습니다.")
            break

def on_proceed_clicked(b):
    """
    진행 버튼 클릭 시 업로드된 파일이 있는지 확인하고, 파일 처리 함수(process_fits_file)를 호출합니다.
    """
    if 'content' in uploaded_file_info:
        print("진행 버튼이 눌렸습니다. 다음 단계로 넘어갑니다.")
        fits_file = io.BytesIO(uploaded_file_info['content'])
        process_fits_file(fits_file)
    else:
        print("파일이 업로드되지 않았습니다. 다시 시도해 주세요.")

def main():
    """
    메인 함수:
    - fits_file_input 변수가 "upload"로 지정된 경우, 사용자가 FITS 파일을 업로드하도록 파일 업로드 위젯과 진행 버튼을 표시합니다.
    - 그렇지 않으면 파일 경로를 직접 사용합니다.
    """
    # fits_file_input 값이 "upload"이면 파일 업로드 위젯을 사용
    fits_file_input = "upload"  # 또는 파일 경로를 직접 지정할 수 있습니다.
    
    if fits_file_input == "upload":
        upload_widget = FileUpload(accept='.fits', multiple=False)
        proceed_button = Button(description="업로드 완료 및 진행")
        
        # 콜백 함수 연결
        upload_widget.observe(on_upload_change, names='value')
        proceed_button.on_click(on_proceed_clicked)
        
        display(VBox([upload_widget, proceed_button]))
    else:
        # 파일 경로를 직접 사용
        process_fits_file(fits_file_input)

# 메인 함수 실행 (Jupyter Notebook에서 위젯이 표시됩니다)
main()